In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('circle_chart_data(20240927)_jihoon.csv')

In [3]:
new_df = df[['date','New_Artist']]

In [12]:
new_df.head()

,date,New_Artist,new_date
0,2023-09-24,악뮤,2023-09-25
1,2023-09-24,악뮤,2023-09-25
2,2023-09-24,"다이나믹 듀오 , 이영지",2023-09-25
3,2023-09-24,뉴진스,2023-09-25
4,2023-09-24,정국,2023-09-25


In [11]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10200 entries, 0 to 10199
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        10200 non-null  datetime64[ns]
 1   New_Artist  10200 non-null  object        
 2   new_date    10200 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 239.2+ KB


In [10]:
# 1. 컬럼을 datetime 형식으로 변환
new_df['date'] = pd.to_datetime(new_df['date'])

# 2. 1일을 더하기
new_df['new_date'] = new_df['date'] + pd.Timedelta(days=1)

C:\Users\user\AppData\Local\Temp\ipykernel_23172\3059192969.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['date'] = pd.to_datetime(new_df['date'])
C:\Users\user\AppData\Local\Temp\ipykernel_23172\3059192969.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['new_date'] = new_df['date'] + pd.Timedelta(days=1)


In [15]:
filtered_df = new_df[~new_df['New_Artist'].str.contains(',')]

In [16]:
filtered_df.head()

,date,New_Artist,new_date
0,2023-09-24,악뮤,2023-09-25
1,2023-09-24,악뮤,2023-09-25
3,2023-09-24,뉴진스,2023-09-25
4,2023-09-24,정국,2023-09-25
5,2023-09-24,뉴진스,2023-09-25


In [17]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9777 entries, 0 to 10199
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        9777 non-null   datetime64[ns]
 1   New_Artist  9777 non-null   object        
 2   new_date    9777 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 305.5+ KB


In [6]:
import os
import sys
import urllib.request
client_id = "nqO1q20si0Nu9yBxeNM0"
client_secret = "JCUXxMt6mk"
url = "https://openapi.naver.com/v1/datalab/search";
body = "{\"startDate\":\"2023-09-25\",\"endDate\":\"2023-10-01\",\"timeUnit\":\"week\",\"keywordGroups\":[{\"groupName\":\"악뮤\",\"keywords\":[\"악뮤\",\"korean\"]}]}";

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{"startDate":"2023-09-25","endDate":"2023-10-01","timeUnit":"week","results":[{"title":"악뮤","keywords":["악뮤","korean"],"data":[{"period":"2023-09-25","ratio":100}]}]}


In [18]:
import os
import sys
import urllib.request
import json
import pandas as pd
from datetime import timedelta

# 데이터프레임 예시 (filtered_df가 있다고 가정)
# df['date']는 datetime 형식으로 변환되었다고 가정

# 네이버 API 클라이언트 정보
clients = [
    {"client_id": "sMr2_6qjZ4bp0mxZoWOp", "client_secret": "icNwNiPkT9"},
    {"client_id": "ndoVgdNtP3oaKB7AFonP", "client_secret": "e7A2QygYPd"},
    {"client_id": "lCA8LFLcRR1cGgcfIesR", "client_secret": "svkDAu7FUq"},
]

client_index = 0  # 현재 사용할 클라이언트 인덱스
request_count = 0  # 요청 수를 추적

# ratio 결과를 저장할 리스트
ratios = []

# API 호출을 위한 함수 정의
def call_naver_api(group_name, start_date, end_date):
    global request_count, client_index

    # 클라이언트 계정이 4900번 호출되면 다음 계정으로 전환
    if request_count >= 4900:
        client_index = (client_index + 1) % len(clients)
        request_count = 0  # 요청 수 초기화

    client_id = clients[client_index]['client_id']
    client_secret = clients[client_index]['client_secret']

    # 네이버 API URL 및 요청 바디 구성
    url = "https://openapi.naver.com/v1/datalab/search"
    body = {
        "startDate": start_date,
        "endDate": end_date,
        "timeUnit": "week",
        "keywordGroups": [
            {
                "groupName": group_name,
                "keywords": [group_name]
            }
        ]
    }

    # API 요청 준비
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    request.add_header("Content-Type", "application/json")

    # API 호출
    response = urllib.request.urlopen(request, data=json.dumps(body).encode("utf-8"))
    rescode = response.getcode()

    # 성공적인 응답일 경우
    if rescode == 200:
        response_body = response.read()
        response_data = json.loads(response_body.decode('utf-8'))
        request_count += 1  # 요청 수 증가
        return response_data
    else:
        print("Error Code:" + str(rescode))
        return None

# 각 행에 대해 API 호출하고 결과 처리
for index, row in filtered_df.iterrows():
    group_name = row['New_Artist']
    new_date = row['date']  # 기존의 date 컬럼을 사용

    end_date = new_date + timedelta(days=7)  # endDate는 new_date + 7

    # 날짜를 yyyy-mm-dd 형식으로 변환
    start_date_str = new_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    # 네이버 API 호출
    api_result = call_naver_api(group_name, start_date_str, end_date_str)

    # API 결과에서 'ratio' 값 추출
    if api_result and 'results' in api_result and len(api_result['results']) > 0:
        ratio_data = api_result['results'][0]['data']
        if len(ratio_data) > 0:
            ratio = ratio_data[0]['ratio']  # 첫 번째 기간의 ratio 값 추출
            ratios.append(ratio)
        else:
            ratios.append(None)  # 데이터가 없는 경우
    else:
        ratios.append(None)  # API 결과가 없을 경우

# 'ratio' 컬럼을 데이터프레임에 추가
filtered_df['ratio'] = ratios


HTTPError: HTTP Error 429: Too Many Requests

In [19]:
import os
import urllib.request
import json
import pandas as pd
from datetime import timedelta
from urllib.error import HTTPError

# 네이버 API 클라이언트 정보
clients = [
    {"client_id": "nqO1q20si0Nu9yBxeNM0", "client_secret": "JCUXxMt6mk"},
    {"client_id": "sMr2_6qjZ4bp0mxZoWOp", "client_secret": "icNwNiPkT9"},
    {"client_id": "ndoVgdNtP3oaKB7AFonP", "client_secret": "e7A2QygYPd"}
]

client_index = 0  # 현재 사용할 클라이언트 인덱스
request_count = 0  # 요청 수를 추적

# API 호출을 위한 함수 정의
def call_naver_api(group_name, start_date, end_date):
    global request_count, client_index

    # 클라이언트 계정이 4900번 호출되면 다음 계정으로 전환
    if request_count >= 4900:
        client_index = (client_index + 1) % len(clients)
        request_count = 0  # 요청 수 초기화
        print(f"Switching to client index {client_index}")

    client_id = clients[client_index]['client_id']
    client_secret = clients[client_index]['client_secret']

    # 네이버 API URL 및 요청 바디 구성
    url = "https://openapi.naver.com/v1/datalab/search"
    body = {
        "startDate": start_date,
        "endDate": end_date,
        "timeUnit": "week",
        "keywordGroups": [
            {
                "groupName": group_name,
                "keywords": [group_name]
            }
        ]
    }

    # API 요청 준비
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    request.add_header("Content-Type", "application/json")

    try:
        # API 호출
        response = urllib.request.urlopen(request, data=json.dumps(body).encode("utf-8"))
        rescode = response.getcode()

        # 성공적인 응답일 경우
        if rescode == 200:
            response_body = response.read()
            response_data = json.loads(response_body.decode('utf-8'))
            request_count += 1  # 요청 수 증가
            return response_data
        else:
            print("Error Code:" + str(rescode))
            return None
    except HTTPError as e:
        # Too Many Requests (429) 에러가 발생하면 다음 클라이언트로 전환
        if e.code == 429:
            print(f"Received 429 Error: Too many requests. Switching to next client...")
            client_index = (client_index + 1) % len(clients)
            request_count = 0  # 요청 수 초기화
            return call_naver_api(group_name, start_date, end_date)  # 다음 클라이언트로 다시 시도
        else:
            print(f"HTTP Error occurred: {e}")
            return None

# 각 행에 대해 API 호출하고 결과 처리 및 데이터프레임에 추가
for index, row in filtered_df.iterrows():
    group_name = row['New_Artist']
    new_date = row['date']  # 기존의 date 컬럼을 사용
    end_date = new_date + timedelta(days=7)  # endDate는 new_date + 7

    # 날짜를 yyyy-mm-dd 형식으로 변환
    start_date_str = new_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    # 네이버 API 호출
    api_result = call_naver_api(group_name, start_date_str, end_date_str)

    # API 결과에서 'ratio' 값 추출
    if api_result and 'results' in api_result and len(api_result['results']) > 0:
        ratio_data = api_result['results'][0]['data']
        if len(ratio_data) > 0:
            ratio = ratio_data[0]['ratio']  # 첫 번째 기간의 ratio 값 추출
        else:
            ratio = None  # 데이터가 없는 경우
    else:
        ratio = None  # API 결과가 없을 경우

    # ratio 값을 즉시 데이터프레임에 추가
    filtered_df.at[index, 'ratio'] = ratio

    # 중간 결과 출력 (테스트 중간 확인용)
    print(f"Processed row {index+1}/{len(filtered_df)}: {group_name}, ratio = {ratio}")

# 결과 출력 (최종 데이터프레임)
print(filtered_df)

# 필요 시 CSV 파일로 저장
filtered_df.to_csv('filtered_df_with_ratio.csv', index=False)


Received 429 Error: Too many requests. Switching to next client...
Processed row 1/9777: 악뮤, ratio = 100
Processed row 2/9777: 악뮤, ratio = 100
Processed row 4/9777: 뉴진스, ratio = 100
Processed row 5/9777: 정국, ratio = 62.65007


C:\Users\user\AppData\Local\Temp\ipykernel_23172\727767298.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.at[index, 'ratio'] = ratio


Processed row 6/9777: 뉴진스, ratio = 100
Processed row 7/9777: 전소미, ratio = 100
Processed row 8/9777: 박재정, ratio = 100
Processed row 9/9777: 화사, ratio = 100
Processed row 10/9777: 아이브, ratio = 63.26285
Processed row 11/9777: 우디, ratio = 100
Processed row 12/9777: 스테이씨, ratio = 100
Processed row 13/9777: 여자 아이들, ratio = 94.67482
Processed row 14/9777: 아이브, ratio = 63.26285
Processed row 15/9777: 르세라핌, ratio = 100
Processed row 16/9777: 뉴진스, ratio = 100
Processed row 17/9777: Lauv, ratio = 100
Processed row 18/9777: 에스파, ratio = 90.7462
Processed row 19/9777: 뉴진스, ratio = 100
Processed row 20/9777: 아이브, ratio = 63.26285
Processed row 21/9777: 뉴진스, ratio = 100
Processed row 22/9777: 로이킴, ratio = 100
Processed row 23/9777: 윤하, ratio = 100
Processed row 24/9777: 빅나티, ratio = 100
Processed row 25/9777: 르세라핌, ratio = 100
Processed row 26/9777: 정국, ratio = 62.65007
Processed row 27/9777: 악뮤, ratio = 100
Processed row 28/9777: 폴 블랑코, ratio = 100
Processed row 29/9777: 라이즈, ratio = 94.23222
Proces

In [8]:
filtered_artists = df[~df['New_Artist'].str.contains('[,\(\)]')]['New_Artist'].unique()
filtered_artists

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
C:\Users\user\AppData\Local\Temp\ipykernel_22676\2365873479.py:1: SyntaxWarning: invalid escape sequence '\('
  filtered_artists = df[~df['New_Artist'].str.contains('[,\(\)]')]['New_Artist'].unique()


array(['악뮤', '뉴진스', '정국', '전소미', '박재정', '화사', '아이브', '우디', '스테이씨',
       '여자 아이들', '르세라핌', 'Lauv', '에스파', '로이킴', '윤하', '빅나티', '폴 블랑코',
       '라이즈', '있지', '성시경', '임영웅', 'DK', '잔나비', '탑현', 'Charlie Puth', '뷔',
       '멜로망스', '세븐틴', '크러쉬', '하이키', '지수', '임한별', '김민석', 'NCT U',
       '이세계 아이돌', '허각', '부석순', '디오', '케이시', '#안녕', '우효', '지민',
       'FIFTY FIFTY', 'NCT DREAM', '방탄소년단', '이무진', '폴킴', '제이세라',
       'Agust D', '테이', '지아', '오마이걸', '10cm', '경서', 'Doja Cat', '던',
       '권은비', '바이브', '너드커넥션', '아이유', '릴러말즈', 'BOYNEXTDOOR', '임재현',
       'YOASOBI', '블랙핑크', '데이식스', '백아', 'imase', '영탁', '주호',
       'Justin Bieber', 'KISS OF LIFE', '거미', '지코', 'Anne-Marie', '죠지',
       '비투비', 'WSG워너비', 'Yyeon', 'ZEROBASEONE', '이채연', 'Sondia', '에일리',
       '노을', '백예린', 'Sam Smith', '허용별', 'Ed Sheeran', '데이먼스 이어', '엔믹스',
       '마크툽', '헤이즈', '김승민', '엠씨더맥스', 'Yuuri', 'BE`O', '태연', 'NCT 127',
       '제니', '김호중', '최유리', 'dori', '온앤오프', '투모로우바이투게더', '쿠기', '이효리', '선미',
       'PLAVE', '빅마마', '양다일', '로꼬', '정